In [28]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import re 
import nltk
from nltk.corpus import wordnet as wn, stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("text.csv")
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [3]:
if df.columns[0] == "Unnamed: 0":
    df.rename(columns={"Unnamed: 0": "index"}, inplace=True)
df.set_index("index", inplace= True)

In [40]:
stop_words = set(stopwords.words('english'))
tokens = []
def normalize_text(text):
    text = text.lower()
    token = text.split()
    token = [word for word in token if word not in stop_words]
    tokens.extend(token)
    text = ' '.join(token)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return text
df['text'] = df['text'].apply(normalize_text)
vc = pd.Series(tokens).value_counts()   # descending by default
words = vc.index.tolist()


In [34]:
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())
    return list(synonyms)

def get_antonyms(word):
    antonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            for ant in lemma.antonyms():
                antonyms.add(ant.name().lower())
    return list(antonyms)

def get_hypernyms(word):
    hypernyms = set()
    for syn in wn.synsets(word):
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                hypernyms.add(lemma.name().lower())
    return list(hypernyms)

def get_hyponyms(word):
    hyponyms = set()
    for syn in wn.synsets(word):
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                hyponyms.add(lemma.name().lower())
    return list(hyponyms)

In [46]:
text = "I feel angry and frustrated"

new_words = text.split()
emotion_related_words = []

for word in new_words:
    emotion_related_words.extend(get_synonyms(new_words))
    emotion_related_words.extend(get_antonyms(new_words))
    emotion_related_words.extend(get_hyponyms(new_words))
    emotion_related_words.extend(get_hypernyms(new_words))

print(set(emotion_related_words))

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
token = df.iloc[1]['text'].split()
token